In [1]:
from os.path import dirname, join


from ipywidgets import interact
import numpy as np
from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
output_notebook()

import pandas as pd
from bokeh.charts import HeatMap, bins, output_notebook, show
from bokeh.layouts import row, widgetbox
from bokeh.models import ColumnDataSource, CustomJS
from bokeh.models.widgets import Slider, Button, DataTable, TableColumn, NumberFormatter, AutocompleteInput
from bokeh.io import curdoc

df = pd.read_csv(join(dirname("/Users/idownard/development/bokeh/examples/app/export_csv/"), 'salary_data.csv'))

source = ColumnDataSource(data=dict())

def update(w=50000):
    #current = df[df['salary'] <= w].dropna()
    current = df[(df['salary'] <= w) & (df['name'].str.contains("K"))].dropna()
    source.data = {
        'name'             : current.name,
        'salary'           : current.salary,
        'years_experience' : current.years_experience,
    }
    t=show(row(controls, table), notebook_handle=True)
    push_notebook(handle=t)


slider = Slider(title="Max Salary", start=10000, end=250000, value=150000, step=1000)
slider.on_change('value', lambda attr, old, new: update(slider.value))

columns = [
    TableColumn(field="name", title="Employee Name"),
    TableColumn(field="salary", title="Income", formatter=NumberFormatter(format="$0,0.00")),
    TableColumn(field="years_experience", title="Experience (years)")
]

data_table = DataTable(source=source, columns=columns, width=800)

autocomp = AutocompleteInput(title="Name", completions=df['name'].tolist())

controls = widgetbox(autocomp,slider)
table = widgetbox(data_table)

#update()
#curdoc().add_root(row(controls, table))
#curdoc().title = "Export CSV"

x=interact(update, w=(10000,100000))



In [2]:
df[df['name'].str.contains("Ki")]

,name,salary,years_experience
11,Kimberly Turcotte,54344.025212,3
115,Russel Kilback,37603.595960,0
195,Santina Kirlin,56615.107970,0


In [3]:
df[(df['salary'] <= 40000) & (df['name'].str.contains("K"))].dropna()

,name,salary,years_experience
115,Russel Kilback,37603.595960,0
151,Karen Stamm,36016.072271,2
206,Emory Kautzer,13567.314696,4
211,Kaela Mertz,29883.780852,5


In [9]:
products=['Debit Card',
          'Personal Credit Card',
          'Business Credit Card',
          'Home Mortgage Loan',
          'Auto Loan',
          'Brokerage Account',
          'Roth IRA',
          '401k',
          'Home Insurance',
          'Automobile Insurance',
          'Medical Insurance',
          'Life Insurance',
          'Cell Phone',
          'Landline'
         ]
#Initializes plotting library and functions for 3D scatter plots 
from pyspark.ml.feature import VectorAssembler
from sklearn.datasets import make_blobs
from sklearn.datasets import make_gaussian_quantiles
from sklearn.datasets import make_classification, make_regression
from sklearn.externals import six
from os.path import dirname, join
from bokeh.layouts import row, widgetbox, column, gridplot
from bokeh.models import ColumnDataSource, CustomJS, FixedTicker
from bokeh.models.widgets import Slider, Button, DataTable, TableColumn, NumberFormatter
from bokeh.io import curdoc
from bokeh.charts import HeatMap, bins, output_notebook, show
from bokeh.palettes import RdYlGn6, RdYlGn9

import pandas as pd
import numpy as np
import argparse
import json
import re
import os
import sys

products=['Debit Card',
          'Personal Credit Card',
          'Business Credit Card',
          'Home Mortgage Loan',
          'Auto Loan',
          'Brokerage Account',
          'Roth IRA',
          '401k',
          'Home Insurance',
          'Automobile Insurance',
          'Medical Insurance',
          'Life Insurance',
          'Cell Phone',
          'Landline'
         ]

def rename_columns(df):
    df = df.copy()
    df.columns = [products[i] for i in df.columns]
    return df

# create an artificial dataset with 3 clusters
X, Y = make_classification(n_samples=100, n_classes=4, n_features=12, n_redundant=0, n_informative=12,
                             scale=1000, n_clusters_per_class=1)
df = pd.DataFrame(X)
# ensure all values are positive (this is needed for our customer 360 use-case)
df = df.abs()
# rename X columns
df = rename_columns(df)
# and add the Y
df['y'] = Y
#df
# split df into cluster groups
grouped = df.groupby(['y'], sort=True)

# compute sums for every column in every group
sums = grouped.sum()

score = []
for x in sums.apply(tuple):
    score.extend(x)
    
data = {
  'persona': list(sums.index) * len(sums.columns),
  'product':  [item for item in list(sums.columns) for i in range(len(sums.index))],
  'score':   score,
}

#hm = HeatMap(data, x='metric', y='players',values='score', title='Fruits', stat=None)
hm = HeatMap(data, x='product', y='persona',values='score', title='Customer Profiles', xlabel='Product',
             ylabel='Persona', legend=False, stat=None, hover_tool=True)
hm.yaxis.ticker=FixedTicker(ticks=range(len(sums.index)))

output_notebook()
show(column(
  gridplot(hm,
           ncols=1, plot_width=800, plot_height=400))
)

Loading BokehJS ...